<a href="https://colab.research.google.com/github/AmBha21/sparta-lab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertModel
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'
print(left_wing_path)

/content/drive/My Drive/sparta lab/transcripts/left_videos.txt


In [ ]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = BertModel.from_pretrained('bert-large-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Ignore this block below

In [ ]:
# testing model
input_text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(input_text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states[0][0])


tensor([-0.3982, -0.5164,  0.1184,  ..., -0.5752, -0.4897,  0.2587])


Setting left wing text variable

In [ ]:
with open(left_wing_path, 'r', encoding='utf-8') as file:
    left_wing_text = file.read()
    # print(left_wing_text[:200])
portion_length = len(left_wing_text) // 400

# Extract the 1/20th portion of the text
smaller_text_portion = left_wing_text[:portion_length]
# print(smaller_text_portion)

he
checked off his equipment
hello my name is Abigail
over the last few years I have emailed
Britain's National healthcare provider
133 times trying to get a doctor's
appointment
this video is about what that was like
why big institutions fall apart and why
they're so difficult to change
as research I've spoken to some of the
nhs's most senior administrators so over
the course of this rational analysis
I'll be revealing some exclusive Insider
information
the video is sponsored by nebula but
after I've paid back the owner of this
laptop I'll be donating most of that
money to charity so you can sit back and
relax as I bring you a calm
dispassionate exploration of the big
questions at the heart of Britain's
National Health Service
[Music]
welcome back
[Music]
[Music]
here's how things are supposed to work
British citizens of all ages have a
right to free health care on the
National Health Service or NHS I was
born in the NHS and both my parents were
doctors my dad used to say when I was a

Making segments for BERT

In [ ]:
def segment_and_tokenize(text):
    # Split the text into segments
    sentences = re.split(r'(?<=[.!?])\s+', text)
    segments = []
    current_segment = ""

    for sentence in sentences:
        if len(tokenizer(current_segment + sentence)['input_ids']) > 512:
            segments.append(current_segment)
            current_segment = sentence
        else:
            current_segment += sentence

    segments.append(current_segment)  # Add the last segment
    # print(segments)

    # Tokenize segments
    tokenized_segments = [tokenizer(segment, truncation=True, padding='max_length', max_length=512) for segment in segments]

    return tokenized_segments

# Example usage
tokenized_segments = segment_and_tokenize(smaller_text_portion)


['', "he\nchecked off his equipment\nhello my name is Abigail\nover the last few years I have emailed\nBritain's National healthcare provider\n133 times trying to get a doctor's\nappointment\nthis video is about what that was like\nwhy big institutions fall apart and why\nthey're so difficult to change\nas research I've spoken to some of the\nnhs's most senior administrators so over\nthe course of this rational analysis\nI'll be revealing some exclusive Insider\ninformation\nthe video is sponsored by nebula but\nafter I've paid back the owner of this\nlaptop I'll be donating most of that\nmoney to charity so you can sit back and\nrelax as I bring you a calm\ndispassionate exploration of the big\nquestions at the heart of Britain's\nNational Health Service\n[Music]\nwelcome back\n[Music]\n[Music]\nhere's how things are supposed to work\nBritish citizens of all ages have a\nright to free health care on the\nNational Health Service or NHS I was\nborn in the NHS and both my parents were\nd